In [1]:
#load files needed
import pandas as pd

In [6]:
allEventsData=pd.read_csv("allEventsData.csv", encoding="utf-8", index_col=0) 
allEventsData['datetest'] = pd.to_datetime(allEventsData['timestamp'])

In [29]:
allEventsData.head()

,id,matchid,outcome,team,teamid,timestamp,type,winner,datetest
0,1543551217,988751,1,NONE,NONE,2018-07-28T15:05:32.59,34,team_1,2018-07-28 15:05:32.590
1,1357993239,988751,1,NONE,NONE,2018-07-28T15:09:55.17,34,team_1,2018-07-28 15:09:55.170
2,1634877849,988751,1,NONE,NONE,2018-07-28T16:00:12.288,32,team_1,2018-07-28 16:00:12.288
3,1097243024,988751,1,NONE,NONE,2018-07-28T16:00:12.288,32,team_1,2018-07-28 16:00:12.288
4,1184172299,988751,1,team_0,t2081,2018-07-28T16:00:13.432,1,team_1,2018-07-28 16:00:13.432


In [5]:
allEventsData.dtypes

id            int64
matchid       int64
outcome       int64
team         object
teamid       object
timestamp    object
type          int64
winner       object
datetest     object
dtype: object

In [8]:
allMatchData = pd.read_csv("allMatchData.csv", encoding="utf-8", index_col=0) 

In [10]:
allMatchData.head()

,matchdata,matchid,team_0_s_accurate_back_zone_pass,team_0_s_accurate_chipped_pass,team_0_s_accurate_corners_intobox,team_0_s_accurate_cross,team_0_s_accurate_cross_nocorner,team_0_s_accurate_flick_on,team_0_s_accurate_freekick_cross,team_0_s_accurate_fwd_zone_pass,...,team_1_s_total_throws,team_1_s_total_yel_card,team_1_s_touches,team_1_s_touches_in_opp_box,team_1_s_unsuccessful_touch,team_1_s_won_contest,team_1_s_won_corners,team_1_s_won_tackle,team_1_teamRef,winner
0,20180728T160000+0100,988751,140.0,17.0,1.0,4.0,3.0,0.0,0.0,151.0,...,18.0,1.0,530.0,24.0,15.0,6.0,9.0,15.0,t2089,team_1
1,20180728T160000+0100,988750,225.0,30.0,0.0,1.0,1.0,1.0,0.0,166.0,...,16.0,1.0,431.0,9.0,8.0,5.0,6.0,12.0,t4950,team_0
2,20180729T160000+0100,988753,114.0,6.0,1.0,2.0,1.0,2.0,0.0,60.0,...,27.0,2.0,928.0,24.0,14.0,6.0,5.0,13.0,t212,team_1
3,20180729T160000+0100,988752,207.0,13.0,0.0,5.0,5.0,2.0,0.0,234.0,...,24.0,4.0,445.0,10.0,12.0,4.0,4.0,12.0,t280,team_0
4,20180729T160000+0100,988754,72.0,12.0,5.0,9.0,4.0,3.0,0.0,128.0,...,32.0,2.0,619.0,26.0,13.0,5.0,3.0,10.0,t5666,team_0


In [11]:
parsedMatches = list(allMatchData['matchid'])

In [12]:
len(parsedMatches)

83

Reforamt Events for ml

In [15]:
eventtypelist = list(allEventsData.type.unique())
print("eventtypes: "+str(len(eventtypelist)))
parsedMatches = list(allMatchData['matchid'])
print("parsedMatches: "+str(len(parsedMatches)))
mlrows = []
gameIndex = 0;
for gameID in parsedMatches:
    print(str(gameIndex)+"start transfrom match: "+str(gameID))
    matchData = allEventsData[allEventsData['matchid']==gameID]
    eventIdList = list(matchData['id'])
    print(str(gameIndex)+"transforming events :"+str(len(eventIdList)))
    eventIndex = 0
    for eventId in eventIdList:
        #get all events before this event  
        if eventIndex%100==0:
            print(str(gameIndex)+" done event "+str(eventIndex))
        row = matchData[(matchData['id']==eventId)]
        currenttimestamp = row['datetest'].values[0]
        toadd = {}
        toadd['team']=row['team'].values[0]    
        toadd['type']=row['type'].values[0]
        toadd['timestamp']=row['timestamp'].values[0]
        toadd['winner']=row['winner'].values[0]
        toadd['teamid']=row['datetest'].values[0]
        toadd['matchid']=gameID
        toadd['id']=eventId
        #for each types for each team the sum
        previousData = matchData[(matchData['matchid']==gameID) \
                                             & (matchData['datetest']<currenttimestamp) \
                                             & (matchData['outcome']==1)]
        for eventtype in eventtypelist:
            #make it for team 1 for testing
            previousRows = previousData[(previousData['team']=='team_0') \
                                            & (previousData['type']==eventtype)]
            toadd['team_0_'+str(eventtype)] = len(previousRows)
            previousRows = previousData[(previousData['team']=='team_1') \
                                            & (previousData['type']==eventtype)]
            toadd['team_1_'+str(eventtype)] = len(previousRows)
        mlrows.append(toadd)
        eventIndex=eventIndex+1
    print(str(gameIndex)+"end transfrom match: "+str(gameID))
    gameIndex = gameIndex+1

eventtypes: 50
parsedMatches: 83
0start transfrom match: 988751
0transforming events :1653
0 done event 0
0 done event 100
0 done event 200
0 done event 300
0 done event 400
0 done event 500
0 done event 600
0 done event 700
0 done event 800
0 done event 900
0 done event 1000
0 done event 1100
0 done event 1200
0 done event 1300
0 done event 1400
0 done event 1500
0 done event 1600
0end transfrom match: 988751
1start transfrom match: 988750
1transforming events :1502
1 done event 0
1 done event 100
1 done event 200
1 done event 300
1 done event 400
1 done event 500
1 done event 600
1 done event 700
1 done event 800
1 done event 900
1 done event 1000
1 done event 1100
1 done event 1200
1 done event 1300
1 done event 1400
1 done event 1500
1end transfrom match: 988750
2start transfrom match: 988753
2transforming events :1746
2 done event 0
2 done event 100
2 done event 200
2 done event 300
2 done event 400
2 done event 500
2 done event 600
2 done event 700
2 done event 800
2 done event 9

In [16]:
eventsMlData = pd.DataFrame.from_dict(mlrows)

In [18]:
eventsMlData.to_csv("eventsMlData.csv", encoding="utf-8") 

In [27]:
eventsMlData['teamid']=allEventsData['teamid']

In [30]:
eventsMlData['outcome']=allEventsData['outcome']

In [31]:
eventsMlData['timestamp']=allEventsData['timestamp']

In [33]:
eventsMlData.head()

,id,matchid,team,team_0_1,team_0_10,team_0_11,team_0_12,team_0_13,team_0_14,team_0_15,...,team_1_7,team_1_70,team_1_74,team_1_79,team_1_8,teamid,timestamp,type,winner,outcome
0,1543551217,988751,NONE,0,0,0,0,0,0,0,...,0,0,0,0,0,NONE,2018-07-28T15:05:32.59,34,team_1,1
1,1357993239,988751,NONE,0,0,0,0,0,0,0,...,0,0,0,0,0,NONE,2018-07-28T15:09:55.17,34,team_1,1
2,1634877849,988751,NONE,0,0,0,0,0,0,0,...,0,0,0,0,0,NONE,2018-07-28T16:00:12.288,32,team_1,1
3,1097243024,988751,NONE,0,0,0,0,0,0,0,...,0,0,0,0,0,NONE,2018-07-28T16:00:12.288,32,team_1,1
4,1184172299,988751,team_0,0,0,0,0,0,0,0,...,0,0,0,0,0,t2081,2018-07-28T16:00:13.432,1,team_1,1


In [34]:
eventsMlData.to_csv("eventsMlData.csv", encoding="utf-8") 

In [35]:
len(eventsMlData)

137699